In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torchvision
from PIL import Image


#use cuda, or not? be prepared for a long wait if you don't have cuda capabilities.
use_cuda = True
#input image. the architectures have been designed for 512x512 colour images
ground_truth_path = 'bunny_512.jpg'
#proportion of pixels to black out.
prop = 0.5
#standard deviation of added noise after each training set
sigma = 1./30
#number of training iterations
num_steps = 25001
#number of steps to take before saving an output image
save_frequency = 250
#where to put the output
output_name = 'output/output'
#choose either 'pixel_shuffle' or 'deconv' as the architecture used.
method = 'pixel_shuffle'

#accept a file path to a jpg, return a torch tensor
def jpg_to_tensor(filepath=ground_truth_path):
    pil = Image.open(ground_truth_path)
    pil_to_tensor = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
    if use_cuda:
        tensor = pil_to_tensor(pil).cuda()
    else:
        tensor = pil_to_tensor(pil)
    return tensor.view([1]+list(tensor.shape))

#accept a torch tensor, convert it to a jpg at a certain path
def tensor_to_jpg(tensor, filename):
    tensor = tensor.view(tensor.shape[1:])
    if use_cuda:
        tensor = tensor.cpu()
    tensor_to_pil = torchvision.transforms.Compose([torchvision.transforms.ToPILImage()])
    pil = tensor_to_pil(tensor)
    pil.save(filename)

#function which zeros out a random proportion of pixels from an image tensor.
def zero_out_pixels(tensor, prop=prop):
    if use_cuda:
        mask = torch.rand([1]+[1] + list(tensor.shape[2:])).cuda()
    else:
        mask = torch.rand([1]+[1] + list(tensor.shape[2:]))
    mask[mask<prop] = 0
    mask[mask!=0] = 1
    mask = mask.repeat(1,3,1,1)
    deconstructed = tensor * mask
    return mask, deconstructed

#define an encoder decoder network with pixel shuffle upsampling
class pixel_shuffle_hourglass(nn.Module):
    def __init__(self):
        super(pixel_shuffle_hourglass, self).__init__()
        self.d_conv_1 = nn.Conv2d(3, 8, 5, stride=2, padding=2)
        self.d_bn_1 = nn.BatchNorm2d(8)

        self.d_conv_2 = nn.Conv2d(8, 16, 5, stride=2, padding=2)
        self.d_bn_2 = nn.BatchNorm2d(16)

        self.d_conv_3 = nn.Conv2d(16, 32, 5, stride=2, padding=2)
        self.d_bn_3 = nn.BatchNorm2d(32)
        self.s_conv_3 = nn.Conv2d(32, 4, 5, stride=1, padding=2)

        self.d_conv_4 = nn.Conv2d(32, 64, 5, stride=2, padding=2)
        self.d_bn_4 = nn.BatchNorm2d(64)
        self.s_conv_4 = nn.Conv2d(64, 4, 5, stride=1, padding=2)

        self.d_conv_5 = nn.Conv2d(64, 128, 5, stride=2, padding=2)
        self.d_bn_5 = nn.BatchNorm2d(128)
        self.s_conv_5 = nn.Conv2d(128, 4, 5, stride=1, padding=2)

        self.d_conv_6 = nn.Conv2d(128, 256, 5, stride=2, padding=2)
        self.d_bn_6 = nn.BatchNorm2d(256)

        self.u_conv_5 = nn.Conv2d(68, 128, 5, stride=1, padding=2)
        self.u_bn_5 = nn.BatchNorm2d(128)

        self.u_conv_4 = nn.Conv2d(36, 64, 5, stride=1, padding=2)
        self.u_bn_4 = nn.BatchNorm2d(64)

        self.u_conv_3 = nn.Conv2d(20, 32, 5, stride=1, padding=2)
        self.u_bn_3 = nn.BatchNorm2d(32)

        self.u_conv_2 = nn.Conv2d(8, 16, 5, stride=1, padding=2)
        self.u_bn_2 = nn.BatchNorm2d(16)

        self.u_conv_1 = nn.Conv2d(4, 16, 5, stride=1, padding=2)
        self.u_bn_1 = nn.BatchNorm2d(16)

        self.out_conv = nn.Conv2d(4, 3, 5, stride=1, padding=2)
        self.out_bn = nn.BatchNorm2d(3)

        
    def forward(self, noise):
        down_1 = self.d_conv_1(noise)
        down_1 = self.d_bn_1(down_1)
        down_1 = F.leaky_relu(down_1)
        
        down_2 = self.d_conv_2(down_1)
        down_2 = self.d_bn_2(down_2)
        down_2 = F.leaky_relu(down_2)

        down_3 = self.d_conv_3(down_2)
        down_3 = self.d_bn_3(down_3)
        down_3 = F.leaky_relu(down_3)
        skip_3 = self.s_conv_3(down_3)

        down_4 = self.d_conv_4(down_3)
        down_4 = self.d_bn_4(down_4)
        down_4 = F.leaky_relu(down_4)
        skip_4 = self.s_conv_4(down_4)

        down_5 = self.d_conv_5(down_4)
        down_5 = self.d_bn_5(down_5)
        down_5 = F.leaky_relu(down_5)
        skip_5 = self.s_conv_5(down_5)

        down_6 = self.d_conv_6(down_5)
        down_6 = self.d_bn_6(down_6)
        down_6 = F.leaky_relu(down_6)

        up_5 = F.pixel_shuffle(down_6, 2)
        up_5 = torch.cat([up_5, skip_5], 1)
        up_5 = self.u_conv_5(up_5)
        up_5 = self.u_bn_5(up_5)
        up_5 = F.leaky_relu(up_5)

        up_4 = F.pixel_shuffle(up_5, 2)
        up_4 = torch.cat([up_4, skip_4], 1)
        up_4 = self.u_conv_4(up_4)
        up_4 = self.u_bn_4(up_4)
        up_4 = F.leaky_relu(up_4)

        up_3 = F.pixel_shuffle(up_4, 2)
        up_3 = torch.cat([up_3, skip_3], 1)
        up_3 = self.u_conv_3(up_3)
        up_3 = self.u_bn_3(up_3)
        up_3 = F.leaky_relu(up_3)

        up_2 = F.pixel_shuffle(up_3, 2)
        up_2 = self.u_conv_2(up_2)
        up_2 = self.u_bn_2(up_2)
        up_2 = F.leaky_relu(up_2)

        up_1 = F.pixel_shuffle(up_2, 2)
        up_1 = self.u_conv_1(up_1)
        up_1 = self.u_bn_1(up_1)
        up_1 = F.leaky_relu(up_1)

        out = F.pixel_shuffle(up_1, 2)
        out = self.out_conv(out)
        out = self.out_bn(out)
        out = F.sigmoid(out)
        return out

#define an encoder decoder network with convolution transpose upsampling.
class deconv_hourglass(nn.Module):
    def __init__(self):
        super(deconv_hourglass, self).__init__()
        self.d_conv_1 = nn.Conv2d(3, 8, 5, stride=2, padding=2)
        self.d_bn_1 = nn.BatchNorm2d(8)

        self.d_conv_2 = nn.Conv2d(8, 16, 5, stride=2, padding=2)
        self.d_bn_2 = nn.BatchNorm2d(16)

        self.d_conv_3 = nn.Conv2d(16, 32, 5, stride=2, padding=2)
        self.d_bn_3 = nn.BatchNorm2d(32)
        self.s_conv_3 = nn.Conv2d(32, 4, 5, stride=1, padding=2)

        self.d_conv_4 = nn.Conv2d(32, 64, 5, stride=2, padding=2)
        self.d_bn_4 = nn.BatchNorm2d(64)
        self.s_conv_4 = nn.Conv2d(64, 4, 5, stride=1, padding=2)

        self.d_conv_5 = nn.Conv2d(64, 128, 5, stride=2, padding=2)
        self.d_bn_5 = nn.BatchNorm2d(128)
        self.s_conv_5 = nn.Conv2d(128, 4, 5, stride=1, padding=2)

        self.d_conv_6 = nn.Conv2d(128, 256, 5, stride=2, padding=2)
        self.d_bn_6 = nn.BatchNorm2d(256)

        self.u_deconv_5 = nn.ConvTranspose2d(256, 124, 4, stride=2, padding=1)
        self.u_bn_5 = nn.BatchNorm2d(128)

        self.u_deconv_4 = nn.ConvTranspose2d(128, 60, 4, stride=2, padding=1)
        self.u_bn_4 = nn.BatchNorm2d(64)

        self.u_deconv_3 = nn.ConvTranspose2d(64, 28, 4, stride=2, padding=1)
        self.u_bn_3 = nn.BatchNorm2d(32)

        self.u_deconv_2 = nn.ConvTranspose2d(32, 16, 4, stride=2, padding=1)
        self.u_bn_2 = nn.BatchNorm2d(16)

        self.u_deconv_2 = nn.ConvTranspose2d(32, 16, 4, stride=2, padding=1)
        self.u_bn_2 = nn.BatchNorm2d(16)

        self.u_deconv_1 = nn.ConvTranspose2d(16, 8, 4, stride=2, padding=1)
        self.u_bn_1 = nn.BatchNorm2d(8)

        self.out_deconv = nn.ConvTranspose2d(8, 3, 4, stride=2, padding=1)        
        self.out_bn = nn.BatchNorm2d(3)

        
    def forward(self, noise):
        down_1 = self.d_conv_1(noise)
        down_1 = self.d_bn_1(down_1)
        down_1 = F.leaky_relu(down_1)
        
        down_2 = self.d_conv_2(down_1)
        down_2 = self.d_bn_2(down_2)
        down_2 = F.leaky_relu(down_2)

        down_3 = self.d_conv_3(down_2)
        down_3 = self.d_bn_3(down_3)
        down_3 = F.leaky_relu(down_3)
        skip_3 = self.s_conv_3(down_3)

        down_4 = self.d_conv_4(down_3)
        down_4 = self.d_bn_4(down_4)
        down_4 = F.leaky_relu(down_4)
        skip_4 = self.s_conv_4(down_4)

        down_5 = self.d_conv_5(down_4)
        down_5 = self.d_bn_5(down_5)
        down_5 = F.leaky_relu(down_5)
        skip_5 = self.s_conv_5(down_5)

        down_6 = self.d_conv_6(down_5)
        down_6 = self.d_bn_6(down_6)
        down_6 = F.leaky_relu(down_6)

        up_5 = self.u_deconv_5(down_6)
        up_5 = torch.cat([up_5, skip_5], 1)
        up_5 = self.u_bn_5(up_5)
        up_5 = F.leaky_relu(up_5)

        up_4 = self.u_deconv_4(up_5)
        up_4 = torch.cat([up_4, skip_4], 1)
        up_4 = self.u_bn_4(up_4)
        up_4 = F.leaky_relu(up_4)

        up_3 = self.u_deconv_3(up_4)
        up_3 = torch.cat([up_3, skip_3], 1)
        up_3 = self.u_bn_3(up_3)
        up_3 = F.leaky_relu(up_3)

        up_2 = self.u_deconv_2(up_3)
        up_2 = self.u_bn_2(up_2)
        up_2 = F.leaky_relu(up_2)

        up_1 = self.u_deconv_1(up_2)
        up_1 = self.u_bn_1(up_1)
        up_1 = F.leaky_relu(up_1)

        out = self.out_deconv(up_1)
        out = self.out_bn(out)
        out = F.sigmoid(out)

        return out

if __name__=='__main__':
    #import image
    truth = jpg_to_tensor(ground_truth_path)
    #deconstruct image
    mask, deconstructed = zero_out_pixels(truth)
    #save the deconstructed image
    tensor_to_jpg(deconstructed, 'deconstructed.jpg')
    #convert the image and mask to variables.
    mask = Variable(mask)
    deconstructed = Variable(deconstructed)

    #input of the network is noise
    if use_cuda:
        noise = Variable(torch.randn(deconstructed.shape).cuda())
    else:
        noise = Variable(torch.randn(deconstructed.shape))

    #initialise the network with the chosen architecture
    if method=='pixel_shuffle':
        net = pixel_shuffle_hourglass()
    elif method=='deconv':
        net = deconv_hourglass()

    #bind the network to the gpu if cuda is enabled
    if use_cuda:
        net.cuda()
    #network optimizer set up
    optimizer = torch.optim.Adam(net.parameters(), lr=1e-4)

    #dummy index to provide names to output files
    save_img_ind = 0
    for step in range(num_steps):
        #get the network output
        output = net(noise)
        #we are only concerned with the output where we have the image available.
        masked_output = output * mask
        # calculate the l2_loss over the masked output and take an optimizer step
        optimizer.zero_grad()
        loss = torch.sum((masked_output - deconstructed)**2)
        loss.backward()
        optimizer.step()
        print('At step {}, loss is {}'.format(step, loss.data.cpu()))
        #every save_frequency steps, save a jpg
        if step % save_frequency == 0:
            tensor_to_jpg(output.data,output_name+'_{}.jpg'.format(save_img_ind))
            save_img_ind += 1
        if use_cuda:
            noise.data += sigma * torch.randn(noise.shape).cuda()
        else:
            noise.data += sigma * torch.randn(noise.shape)

    #clean up any mess we're leaving on the gpu
    if use_cuda:
        torch.cuda.empty_cache()


At step 0, loss is 
 17260.1191
[torch.FloatTensor of size 1]

At step 1, loss is 
 16581.7266
[torch.FloatTensor of size 1]

At step 2, loss is 
 15966.0391
[torch.FloatTensor of size 1]

At step 3, loss is 
 15292.6875
[torch.FloatTensor of size 1]

At step 4, loss is 
 14580.5615
[torch.FloatTensor of size 1]

At step 5, loss is 
 13923.6777
[torch.FloatTensor of size 1]

At step 6, loss is 
 13387.6260
[torch.FloatTensor of size 1]

At step 7, loss is 
 12954.3428
[torch.FloatTensor of size 1]

At step 8, loss is 
 12620.9297
[torch.FloatTensor of size 1]

At step 9, loss is 
 12356.8174
[torch.FloatTensor of size 1]

At step 10, loss is 
 12137.7891
[torch.FloatTensor of size 1]

At step 11, loss is 
 11956.1543
[torch.FloatTensor of size 1]

At step 12, loss is 
 11800.1533
[torch.FloatTensor of size 1]

At step 13, loss is 
 11664.3682
[torch.FloatTensor of size 1]

At step 14, loss is 
 11545.3564
[torch.FloatTensor of size 1]

At step 15, loss is 
 11439.2148
[torch.FloatTenso

At step 131, loss is 
 8767.9482
[torch.FloatTensor of size 1]

At step 132, loss is 
 8756.9580
[torch.FloatTensor of size 1]

At step 133, loss is 
 8746.0547
[torch.FloatTensor of size 1]

At step 134, loss is 
 8735.3164
[torch.FloatTensor of size 1]

At step 135, loss is 
 8723.1602
[torch.FloatTensor of size 1]

At step 136, loss is 
 8711.5244
[torch.FloatTensor of size 1]

At step 137, loss is 
 8699.7773
[torch.FloatTensor of size 1]

At step 138, loss is 
 8686.8809
[torch.FloatTensor of size 1]

At step 139, loss is 
 8674.3262
[torch.FloatTensor of size 1]

At step 140, loss is 
 8660.5342
[torch.FloatTensor of size 1]

At step 141, loss is 
 8647.6797
[torch.FloatTensor of size 1]

At step 142, loss is 
 8634.2773
[torch.FloatTensor of size 1]

At step 143, loss is 
 8620.8770
[torch.FloatTensor of size 1]

At step 144, loss is 
 8606.7979
[torch.FloatTensor of size 1]

At step 145, loss is 
 8592.8135
[torch.FloatTensor of size 1]

At step 146, loss is 
 8578.7246
[torch.

At step 261, loss is 
 6537.4731
[torch.FloatTensor of size 1]

At step 262, loss is 
 6529.0698
[torch.FloatTensor of size 1]

At step 263, loss is 
 6520.9111
[torch.FloatTensor of size 1]

At step 264, loss is 
 6512.5986
[torch.FloatTensor of size 1]

At step 265, loss is 
 6504.6450
[torch.FloatTensor of size 1]

At step 266, loss is 
 6497.2192
[torch.FloatTensor of size 1]

At step 267, loss is 
 6489.0200
[torch.FloatTensor of size 1]

At step 268, loss is 
 6481.4692
[torch.FloatTensor of size 1]

At step 269, loss is 
 6473.9580
[torch.FloatTensor of size 1]

At step 270, loss is 
 6465.6704
[torch.FloatTensor of size 1]

At step 271, loss is 
 6457.4312
[torch.FloatTensor of size 1]

At step 272, loss is 
 6450.4238
[torch.FloatTensor of size 1]

At step 273, loss is 
 6443.0674
[torch.FloatTensor of size 1]

At step 274, loss is 
 6435.1592
[torch.FloatTensor of size 1]

At step 275, loss is 
 6428.1655
[torch.FloatTensor of size 1]

At step 276, loss is 
 6420.6274
[torch.

At step 395, loss is 
 5827.2031
[torch.FloatTensor of size 1]

At step 396, loss is 
 5823.3462
[torch.FloatTensor of size 1]

At step 397, loss is 
 5819.3999
[torch.FloatTensor of size 1]

At step 398, loss is 
 5815.1611
[torch.FloatTensor of size 1]

At step 399, loss is 
 5811.8062
[torch.FloatTensor of size 1]

At step 400, loss is 
 5807.9243
[torch.FloatTensor of size 1]

At step 401, loss is 
 5803.4404
[torch.FloatTensor of size 1]

At step 402, loss is 
 5799.7305
[torch.FloatTensor of size 1]

At step 403, loss is 
 5795.9707
[torch.FloatTensor of size 1]

At step 404, loss is 
 5792.3867
[torch.FloatTensor of size 1]

At step 405, loss is 
 5789.0786
[torch.FloatTensor of size 1]

At step 406, loss is 
 5784.8198
[torch.FloatTensor of size 1]

At step 407, loss is 
 5780.7842
[torch.FloatTensor of size 1]

At step 408, loss is 
 5776.7593
[torch.FloatTensor of size 1]

At step 409, loss is 
 5772.9922
[torch.FloatTensor of size 1]

At step 410, loss is 
 5769.5293
[torch.

At step 528, loss is 
 5243.1455
[torch.FloatTensor of size 1]

At step 529, loss is 
 5237.5610
[torch.FloatTensor of size 1]

At step 530, loss is 
 5231.2446
[torch.FloatTensor of size 1]

At step 531, loss is 
 5225.3315
[torch.FloatTensor of size 1]

At step 532, loss is 
 5218.5684
[torch.FloatTensor of size 1]

At step 533, loss is 
 5213.6240
[torch.FloatTensor of size 1]

At step 534, loss is 
 5206.4609
[torch.FloatTensor of size 1]

At step 535, loss is 
 5199.8022
[torch.FloatTensor of size 1]

At step 536, loss is 
 5193.4771
[torch.FloatTensor of size 1]

At step 537, loss is 
 5187.6055
[torch.FloatTensor of size 1]

At step 538, loss is 
 5180.9111
[torch.FloatTensor of size 1]

At step 539, loss is 
 5174.5747
[torch.FloatTensor of size 1]

At step 540, loss is 
 5168.1948
[torch.FloatTensor of size 1]

At step 541, loss is 
 5161.7812
[torch.FloatTensor of size 1]

At step 542, loss is 
 5155.6880
[torch.FloatTensor of size 1]

At step 543, loss is 
 5149.8223
[torch.

...

At step 24424, loss is 
 136.5021
[torch.FloatTensor of size 1]

At step 24425, loss is 
 137.4929
[torch.FloatTensor of size 1]

At step 24426, loss is 
 137.8785
[torch.FloatTensor of size 1]

At step 24427, loss is 
 138.6870
[torch.FloatTensor of size 1]

At step 24428, loss is 
 137.2626
[torch.FloatTensor of size 1]

At step 24429, loss is 
 135.7926
[torch.FloatTensor of size 1]

At step 24430, loss is 
 134.5497
[torch.FloatTensor of size 1]

At step 24431, loss is 
 134.5600
[torch.FloatTensor of size 1]

At step 24432, loss is 
 135.1366
[torch.FloatTensor of size 1]

At step 24433, loss is 
 136.0728
[torch.FloatTensor of size 1]

At step 24434, loss is 
 137.1787
[torch.FloatTensor of size 1]

At step 24435, loss is 
 136.6941
[torch.FloatTensor of size 1]

At step 24436, loss is 
 135.7228
[torch.FloatTensor of size 1]

At step 24437, loss is 
 134.8279
[torch.FloatTensor of size 1]

At step 24438, loss is 
 134.6324
[torch.FloatTensor of size 1]

At step 24439, loss is 
 

At step 24553, loss is 
 134.0620
[torch.FloatTensor of size 1]

At step 24554, loss is 
 134.0814
[torch.FloatTensor of size 1]

At step 24555, loss is 
 134.3845
[torch.FloatTensor of size 1]

At step 24556, loss is 
 134.5380
[torch.FloatTensor of size 1]

At step 24557, loss is 
 134.9667
[torch.FloatTensor of size 1]

At step 24558, loss is 
 135.1917
[torch.FloatTensor of size 1]

At step 24559, loss is 
 135.5441
[torch.FloatTensor of size 1]

At step 24560, loss is 
 135.6768
[torch.FloatTensor of size 1]

At step 24561, loss is 
 135.7896
[torch.FloatTensor of size 1]

At step 24562, loss is 
 135.4905
[torch.FloatTensor of size 1]

At step 24563, loss is 
 135.3714
[torch.FloatTensor of size 1]

At step 24564, loss is 
 135.0157
[torch.FloatTensor of size 1]

At step 24565, loss is 
 134.8575
[torch.FloatTensor of size 1]

At step 24566, loss is 
 134.3942
[torch.FloatTensor of size 1]

At step 24567, loss is 
 134.2516
[torch.FloatTensor of size 1]

At step 24568, loss is 
 

At step 24684, loss is 
 135.0515
[torch.FloatTensor of size 1]

At step 24685, loss is 
 135.1292
[torch.FloatTensor of size 1]

At step 24686, loss is 
 134.4221
[torch.FloatTensor of size 1]

At step 24687, loss is 
 134.2445
[torch.FloatTensor of size 1]

At step 24688, loss is 
 134.1694
[torch.FloatTensor of size 1]

At step 24689, loss is 
 134.2779
[torch.FloatTensor of size 1]

At step 24690, loss is 
 134.6107
[torch.FloatTensor of size 1]

At step 24691, loss is 
 134.4160
[torch.FloatTensor of size 1]

At step 24692, loss is 
 134.4753
[torch.FloatTensor of size 1]

At step 24693, loss is 
 133.9580
[torch.FloatTensor of size 1]

At step 24694, loss is 
 133.7620
[torch.FloatTensor of size 1]

At step 24695, loss is 
 134.0376
[torch.FloatTensor of size 1]

At step 24696, loss is 
 134.2674
[torch.FloatTensor of size 1]

At step 24697, loss is 
 134.5550
[torch.FloatTensor of size 1]

At step 24698, loss is 
 134.6065
[torch.FloatTensor of size 1]

At step 24699, loss is 
 

At step 24813, loss is 
 133.8720
[torch.FloatTensor of size 1]

At step 24814, loss is 
 133.8997
[torch.FloatTensor of size 1]

At step 24815, loss is 
 133.9393
[torch.FloatTensor of size 1]

At step 24816, loss is 
 134.0776
[torch.FloatTensor of size 1]

At step 24817, loss is 
 134.0480
[torch.FloatTensor of size 1]

At step 24818, loss is 
 134.0867
[torch.FloatTensor of size 1]

At step 24819, loss is 
 134.3443
[torch.FloatTensor of size 1]

At step 24820, loss is 
 134.7379
[torch.FloatTensor of size 1]

At step 24821, loss is 
 135.3040
[torch.FloatTensor of size 1]

At step 24822, loss is 
 136.3565
[torch.FloatTensor of size 1]

At step 24823, loss is 
 137.1710
[torch.FloatTensor of size 1]

At step 24824, loss is 
 138.4812
[torch.FloatTensor of size 1]

At step 24825, loss is 
 138.6298
[torch.FloatTensor of size 1]

At step 24826, loss is 
 138.5099
[torch.FloatTensor of size 1]

At step 24827, loss is 
 136.5650
[torch.FloatTensor of size 1]

At step 24828, loss is 
 

At step 24940, loss is 
 133.6114
[torch.FloatTensor of size 1]

At step 24941, loss is 
 133.6281
[torch.FloatTensor of size 1]

At step 24942, loss is 
 133.7076
[torch.FloatTensor of size 1]

At step 24943, loss is 
 133.9284
[torch.FloatTensor of size 1]

At step 24944, loss is 
 133.9431
[torch.FloatTensor of size 1]

At step 24945, loss is 
 133.9975
[torch.FloatTensor of size 1]

At step 24946, loss is 
 133.8559
[torch.FloatTensor of size 1]

At step 24947, loss is 
 133.7341
[torch.FloatTensor of size 1]

At step 24948, loss is 
 133.5246
[torch.FloatTensor of size 1]

At step 24949, loss is 
 133.3559
[torch.FloatTensor of size 1]

At step 24950, loss is 
 133.3406
[torch.FloatTensor of size 1]

At step 24951, loss is 
 133.2846
[torch.FloatTensor of size 1]

At step 24952, loss is 
 133.3108
[torch.FloatTensor of size 1]

At step 24953, loss is 
 133.3393
[torch.FloatTensor of size 1]

At step 24954, loss is 
 133.4753
[torch.FloatTensor of size 1]

At step 24955, loss is 
 